In [1]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from numpy.lib.stride_tricks import as_strided
from scipy.signal import convolve2d

from tf_hog import tf_hog_descriptor

from PIL import Image
%matplotlib inline

plt.rcParams['image.cmap'] = 'viridis'

In [3]:
def read_images(folder):
    fnames = [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]
    images = [np.array(Image.open(os.path.join(folder, f))) for f in fnames]
    return np.array(images)

data = read_images('cifar/')

In [4]:
import numpy as np
import tensorflow as tf
from tf_filters import tf_deriv


def tf_select_by_idx(a, idx):
    return tf.select(tf.equal(idx, 2), 
                     a[:,:,:,2], 
                     tf.select(tf.equal(idx, 1), 
                               a[:,:,:,1], 
                               a[:,:,:,0]))


def tf_hog_descriptor(images, cell_size = 8, block_size = 2, block_stride = 1, n_bins = 9,
                      grayscale = False, oriented = False):

    batch_size, height, width, depth = images.shape
    half_pi = tf.constant(np.pi/2, name="pi_half")
    eps = tf.constant(1e-6, name="eps")
    scale_factor = tf.constant(np.pi * n_bins * 0.99999, name="scale_factor")
    
    img = tf.constant(images, name="ImgBatch", dtype=tf.float32)

    # gradients
    if grayscale:
        gray = tf.image.rgb_to_grayscale(img, name="ImgGray")
        grad = tf_deriv(gray)
    else:
        grad = tf_deriv(img)
    g_x = grad[:,:,:,0::2]
    g_y = grad[:,:,:,1::2]
    
    # maximum norm gradient selection
    g_norm = tf.sqrt(tf.square(g_x) + tf.square(g_y), "GradNorm")
    idx    = tf.argmax(g_norm, 3)
    
    g_norm = tf.expand_dims(tf_select_by_idx(g_norm, idx), -1)
    g_x    = tf.expand_dims(tf_select_by_idx(g_x,    idx), -1)
    g_y    = tf.expand_dims(tf_select_by_idx(g_y,    idx), -1)

    # orientation and binning
    if oriented:
        # atan2 implementation needed 
        # lots of conditional indexing required
        raise NotImplementedError("`oriented` gradient not supported yet")
    else:
        g_dir = tf.atan(g_y / (g_x + eps)) + half_pi
        g_bin = tf.to_int32(g_dir / scale_factor, name="Bins")  

    # cells partitioning
    cell_norm = tf.space_to_depth(g_norm, cell_size, name="GradCells")
    cell_bins = tf.space_to_depth(g_bin,  cell_size, name="BinsCells")

    # cells histograms
    hist = list()
    zero = tf.zeros(cell_bins.get_shape()) 
    for i in range(n_bins):
        mask = tf.equal(cell_bins, tf.constant(i, name="%i"%i))
        hist.append(tf.reduce_sum(tf.select(mask, cell_norm, zero), 3))
    hist = tf.transpose(tf.pack(hist), [1,2,3,0], name="Hist")

    # blocks partitioning
    block_hist = tf.extract_image_patches(hist, 
                                          ksizes  = [1, block_size, block_size, 1], 
                                          strides = [1, block_stride, block_stride, 1], 
                                          rates   = [1, 1, 1, 1], 
                                          padding = 'VALID',
                                          name    = "BlockHist")

    # block normalization
    block_hist = tf.nn.l2_normalize(block_hist, 3, epsilon=1.0)
    
    # HOG descriptor
    hog_descriptor = tf.reshape(block_hist, 
                                [int(block_hist.get_shape()[0]), 
                                     int(block_hist.get_shape()[1]) * \
                                     int(block_hist.get_shape()[2]) * \
                                     int(block_hist.get_shape()[3])], 
                                 name='HOGDescriptor')

    return hog_descriptor

In [4]:
import time

tf.reset_default_graph()
init_op = tf.initialize_all_variables()
X = np.repeat(np.repeat(data[:10000], 4, axis=1), 2, axis=2)

t1 = time.time()
hog = tf_hog_descriptor(X)
with tf.Session() as sess:
    summary_writer = tf.train.SummaryWriter('tf_logs/', sess.graph)
    sess.run(init_op)
    t2 = time.time()
    g = hog.eval()
    print(g.shape)
t3 = time.time()

print('Total time: %.3f sec' % (t3 - t1))
print('Session management time: %.3f sec' % (t2 - t1))
print('Evaluation time: %.3f' % (t3 - t2))

(10000, 3780)
Total time: 131.022 sec
Session management time: 11.220 sec
Evaluation time: 119.803


In [ ]:
import cv2 as cv
import time

k = (1024-64) // 8 * (768-128) // 8
X = np.random.randint(0,256, (k,128,64,3), dtype=np.uint8)

t1 = time.time()
hog = cv.HOGDescriptor()
for x in X:
    hog.compute(x)
t2 = time.time()

print('Total time: %.3f sec' % (t2 - t1))

In [ ]:
from scipy.signal import convolve2d

def np_hog(a):
    n_bins = 9

    half_pi = np.pi * 0.5
    eps     = 1e-6
    scale_factor = n_bins / np.pi * 0.99999

    def batch_to_depth(a):    
        b = np.transpose(a, [1,2,3,0])
        b = np.reshape(b, [b.shape[0], b.shape[1], b.shape[2]*b.shape[3]])
        return b


    def depth_to_batch(b, batch_size):
        c = np.reshape(b, [b.shape[0], b.shape[1], b.shape[2] // batch_size, batch_size])
        c = np.transpose(c, [3,0,1,2])
        return c

    b = batch_to_depth(a)

    g_x = np.zeros_like(b)
    g_y = np.zeros_like(b)
    for i in range(b.shape[2]):
        g_x[:,:,i] = convolve2d(b[:,:,i], [[-1,0,1]], mode='same', boundary='symm')
        g_y[:,:,i] = convolve2d(b[:,:,i], [[-1],[0],[1]], mode='same', boundary='symm')

    g_x = depth_to_batch(g_x, a.shape[0])
    g_y = depth_to_batch(g_y, a.shape[0])

    g_norm = np.sqrt(np.square(g_x) + np.square(g_y))
    idx    = np.argmax(g_norm, axis = -1)
    mask   = idx[:,:,:,np.newaxis] == np.arange(a.shape[-1])

    g_norm = g_norm[mask].reshape(a.shape[0], a.shape[1], a.shape[2], 1)
    g_x    = g_x[mask].reshape(a.shape[0], a.shape[1], a.shape[2], 1)
    g_y    = g_y[mask].reshape(a.shape[0], a.shape[1], a.shape[2], 1)

    g_dir = np.arctan(g_y / (g_x + eps)) + half_pi
    g_bin = np.int32(g_dir * scale_factor)

    def extract_patches(a, sizes, strides):
        stride_x = strides[0]
        stride_y = strides[1]
        win_x = sizes[0]
        win_y = sizes[1]

        size_t = a.dtype.itemsize
        batch_n, height_n, width_n, channels_n = a.shape

        height_m = (height_n - win_y) // stride_y + 1
        width_m  = (width_n  - win_x) // stride_x + 1

        strides = ( size_t * channels_n * width_n * height_n, size_t * channels_n * width_n * stride_y,
                    size_t * channels_n * stride_x, size_t * channels_n * width_n, size_t * channels_n, size_t )

        shape = ( batch_n, height_m, width_n // stride_x, win_y, win_x, channels_n )

        b = as_strided(a, strides = strides, shape = shape)[:, :, :width_m, :, :, :]
        b = b.reshape(b.shape[0], b.shape[1], b.shape[2], b.shape[3]*b.shape[4]*b.shape[5])
        return b

    cell_bins = extract_patches(g_bin,  [8, 8], [8, 8])
    cell_grad = extract_patches(g_norm, [8, 8], [8, 8])

    hist = list()
    tmp = np.ma.masked_array(np.zeros_like(cell_bins))
    for i in range(n_bins):
        mask = cell_bins != i
        grad_masked = np.ma.masked_array(cell_grad, mask)
        hist.append(np.ma.sum(grad_masked, axis = 3).data)
    hist = np.stack(hist, axis = -1)

    block_hist = extract_patches(hist, [2,2], [1,1])
    k = np.sqrt(np.sum(np.square(block_hist), axis = -1))
    k = np.broadcast_to(k, (block_hist.shape[3], k.shape[0], k.shape[1], k.shape[2]))
    k = np.transpose(k, [1,2,3,0])

    descriptor = np.reshape(block_hist / k, (block_hist.shape[0], 
                                             block_hist.shape[1]*block_hist.shape[2]*block_hist.shape[3]))
    return descriptor